In [40]:
%pip install azure.cosmos azure-identity azure-keyvault-secrets python-dotenv pymongo

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import azure.cosmos
from azure.cosmos.partition_key import PartitionKey
from azure.identity import ClientSecretCredential, AzureCliCredential
from azure.keyvault.secrets import SecretClient
from pymongo import MongoClient
import os

/Users/aidan/Library/Python/3.8/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from dotenv import load_dotenv
load_dotenv("/Users/aidan/Documents/proj_beta/.env")

True

Authenticate and Retrieve Secrets from Azure Key Vault

In [3]:
# Get credentials from environment variables
my_secret_key = os.getenv("SECRET_NAME")

# Authenticate using the client secret credential
credential = AzureCliCredential()
key_vault_url = os.getenv("AZURE_KEY_VAULT_URL")
secret_client = SecretClient(vault_url=key_vault_url, credential=credential)

# Retrieve a secret by its name
secret = secret_client.get_secret(my_secret_key)

Create a new instance of CosmosClient.

In [4]:
# Get the connection string from Azure Key Vault or environment
cosmos_mongo_connection_string = secret.value #os.getenv("COSMOS_MONGO_CONNECTION_STRING")  # Or fetch from Key Vault

# Initialize the MongoDB client
client = MongoClient(cosmos_mongo_connection_string)

# Test connection
try:
    # List databases
    databases = client.list_database_names()
    print("Databases in Cosmos DB (MongoDB API):", databases)
except Exception as e:
    print("Failed to connect to Cosmos DB:", e)

/var/folders/n2/f841nn_95r1dcmhn6z7l1_9w0000gp/T/ipykernel_24528/539981733.py:5: UserWarning: You appear to be connected to a CosmosDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/cosmosdb
  client = MongoClient(cosmos_mongo_connection_string)


Databases in Cosmos DB (MongoDB API): ['projbeta-database']


Create collection in Database

In [11]:
db = client.get_database('projbeta-database')
collections = db.list_collection_names()
db.drop_collection("demo")
collection = db.create_collection("demo")

In [12]:

result = collection.insert_many([
  { "year": '2019', "temperature": 14.2, "rainfall": 800, "emissions": 350 },
  { "year": '2020', "temperature": 14.8, "rainfall": 750, "emissions": 320 },
  { "year": '2021', "temperature": 15.1, "rainfall": 720, "emissions": 310 },
  { "year": '2022', "temperature": 15.4, "rainfall": 680, "emissions": 305 },
])

In [28]:
documents = db.demo.find()

# Print out each document
for document in documents:
    print(document)


{'_id': ObjectId('6756ff3c91fef2db3727aa65'), 'year': '2019', 'temperature': 14.2, 'rainfall': 800, 'emissions': 350}
{'_id': ObjectId('6756ff3c91fef2db3727aa66'), 'year': '2020', 'temperature': 14.8, 'rainfall': 750, 'emissions': 320}
{'_id': ObjectId('6756ff3c91fef2db3727aa67'), 'year': '2021', 'temperature': 15.1, 'rainfall': 720, 'emissions': 310}
{'_id': ObjectId('6756ff3c91fef2db3727aa68'), 'year': '2022', 'temperature': 15.4, 'rainfall': 680, 'emissions': 305}
